In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [7]:
# Set the paths to your training and test data directories
# The images should be in seperate folders labelled with their corresponding names
train_data_dir = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Train" #Edit path to Training image
test_data_dir = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Test" #Edit path to Test images

# Define the image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

In [8]:
#Preprocess
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 75 images belonging to 5 classes.


In [9]:
#test data preprocess
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 75 images belonging to 5 classes.


In [10]:
# Load ResNet50 model 
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(img_width, img_height, 3)
)

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

num_classes = len(train_generator.class_indices)
output = Dense(num_classes, activation='softmax')(x)

In [11]:
# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)


In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=15,  # Increase the number of epochs if needed
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr])


# Save the model
model.save('ResNet50_model.h5')

Epoch 1/15
2/2 [==============================] - 24s 5s/step - loss: 2.5186 - accuracy: 0.2326 - val_loss: 3.6434 - val_accuracy: 0.2188 - lr: 0.0010
Epoch 2/15
2/2 [==============================] - 14s 8s/step - loss: 2.2466 - accuracy: 0.3438 - val_loss: 3.7070 - val_accuracy: 0.1875 - lr: 0.0010
Epoch 3/15
2/2 [==============================] - 15s 8s/step - loss: 1.1278 - accuracy: 0.6250 - val_loss: 3.1526 - val_accuracy: 0.1875 - lr: 0.0010
Epoch 4/15
2/2 [==============================] - 16s 10s/step - loss: 0.7566 - accuracy: 0.7188 - val_loss: 2.9868 - val_accuracy: 0.2188 - lr: 0.0010
Epoch 5/15
2/2 [==============================] - 16s 12s/step - loss: 0.4056 - accuracy: 0.8605 - val_loss: 2.5275 - val_accuracy: 0.2188 - lr: 0.0010
Epoch 6/15
2/2 [==============================] - 16s 7s/step - loss: 0.4137 - accuracy: 0.8605 - val_loss: 3.0620 - val_accuracy: 0.2188 - lr: 0.0010
Epoch 7/15
2/2 [==============================] - 16s 7s/step - loss: 0.4294 - accuracy: 0.9

In [13]:
loaded_model = tf.keras.models.load_model('ResNet50_model.h5')


# Set the path to the folder containing the images
folder_path = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Test\Tom Cruise"


for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    
    # Read and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)         
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.resnet50.preprocess_input(image)
    
    # Make predictions
    pred = loaded_model.predict(image)
    pred_label = np.argmax(pred)
    
    # Map label index to person name
    class_labels = train_generator.class_indices
    person_label = {v: k for k, v in class_labels.items()}
    pred_person = person_label[pred_label]
        
    print('Predicted Person:', pred_person)


1/1 [==============================] - 1s 712ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 68ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 81ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 79ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 79ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 78ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 83ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 82ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 78ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 92ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 90ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 82ms/step
Predicted Person: Keira Knightley
